In [7]:
from cProfile import label
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from scipy.signal import savgol_filter
from scipy import interpolate
from mpl_toolkits.mplot3d import axes3d

In [25]:
# Load dataset
data = np.loadtxt("data_accel.txt").T

time = data[0]
time = (time - time[0])*1e-6
position = data[1:8, :]
control_torque = data[8:15, :]
measured_torque = data[15:22, :]
mass = data[22:, :]

# Filter out noise and estimate position derivatives
measured_torque = savgol_filter(measured_torque, 111, 3)

velocity = savgol_filter(position, window_length=111, polyorder=2, deriv=1, delta=1e-3)
acceleration = savgol_filter(position, window_length=351, polyorder=2, deriv=2, delta=1e-3)

# Estimate model torque from acceleration and mass matrix
est_torque = np.zeros_like(control_torque)*np.nan
for i in range((est_torque.shape[1])):
    new_mass = mass[:, i].reshape((7,7))
    est_torque[:, i] = new_mass@acceleration[:, i]

# Resample dataset at lower frequency and regular spaced time grid
f_position = interpolate.interp1d(time, position, axis=1)
f_velocity = interpolate.interp1d(time, velocity, axis=1)
f_acceleration = interpolate.interp1d(time, acceleration, axis=1)
f_measured_torque = interpolate.interp1d(time, measured_torque, axis=1)
f_control_torque = interpolate.interp1d(time, control_torque, axis=1)
f_est_torque = interpolate.interp1d(time, est_torque, axis=1)

dT = 50e-3
time = np.arange(time[0], time[-1], dT)

position = f_position(time)
velocity = f_velocity(time)
acceleration = f_acceleration(time)
measured_torque = f_measured_torque(time)
control_torque = f_control_torque(time)
est_torque = f_est_torque(time)

torque_error = est_torque - control_torque

In [ ]:
# Time serie
fig, axs = plt.subplots(4, 2)
for i, ax in enumerate(axs.ravel()[:-1]):
    ax.plot(time, control_torque[i, :], label = "Control")
    ax.plotorque_errort(time, measured_torque[i, :], label="Measured")
    ax.plot(time, est_torque[i, :], label="Estimated")

    ax.legend(loc='upper right', prop={'size': 6})
    # ax.set(ylabel="joint {} [degree]".format(i))
fig.suptitle("Joint torques [N.m] over time [s]")

In [27]:
fig = make_subplots(rows=2, cols=4,
    specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}],
           [{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]])

for i in range(7):

    marker_data = go.Scatter3d(
        x=position[i, :], 
        y=velocity[i, :], 
        z= torque_error[i, :],
        marker=go.scatter3d.Marker(size=1), 
        opacity=0.8, 
        mode='markers')
    fig.add_trace(marker_data, row=int(np.floor(i/4))+1, col=i%4 +1)

    # fig=go.Figure(data=marker_data)
fig.update_layout(
    title_text='torque error',
    height=800,
    width=1200,
    scene = dict(
        xaxis_title='position',
        yaxis_title='velocity',
        zaxis_title='torque'))
fig.show()

### Let's try learning this dataset

In [47]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split
import torch.optim as optim

class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(14, 2*14),
      nn.ReLU(),
      nn.Linear(2*14, 2*14),
      nn.ReLU(),
      nn.Linear(2*14, 7)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [49]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

In [48]:
x_tensor = torch.from_numpy(np.concatenate((position, velocity)).T).float()
y_tensor = torch.from_numpy(torque_error.T).float()

train_ratio = 0.8
dataset_size = position.shape[1]
train_size = int(dataset_size*train_ratio)

dataset = TensorDataset(x_tensor, y_tensor)
train_dataset, val_dataset = random_split(dataset, [train_size, dataset_size-train_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=10)



In [40]:
model = MLP().to(device)

loss_fn = nn.MSELoss(reduction='mean')
# optimizer = optim.SGD(model.parameters(), lr=1e-1)
optimizer = optim.Adam(model.parameters(),lr=0.013)

scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9995)

losses = []
val_losses = []
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(1000):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            model.eval()

            yhat = model(x_val)
            val_loss = loss_fn(y_val, yhat)
            val_losses.append(val_loss.item())
    
    scheduler.step()


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [45]:
device

'cpu'